In [1]:
import os
import albumentations as A
import matplotlib.pyplot as plt
from collections import deque
from PIL import Image
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm

import torch

from model import CNN
from dataset import PlantDataset

DATA_PATH = '../data/'

In [11]:
config = dict(
	batch_size=16,
	dataset='Plant',
	image_height=480, # 480, 960
	image_width=640, # 640, 1280
)

resize = A.Compose(
	[
		A.Resize(height=config['image_height'], width=config['image_width']),
	],
)

In [12]:
client_directories = ['A1', 'A2', 'A4', 'A6', 'B1', 'B2', 'B3', 'B4']

for client_dir in client_directories:
	dataset = PlantDataset(set_dir='valid', client_dir=client_dir, transform=resize)
	print(client_dir, len(dataset))

A1 26
A2 25
A4 26
A6 26
B1 25
B2 26
B3 26
B4 14


In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

In [31]:
print(type(model.state_dict()))

print(len(model.state_dict()))

for aaa in model.state_dict(model.state_dict()):
	print(aaa)


<class 'collections.OrderedDict'>
41
layer1.0.weight
layer1.0.bias
layer1.1.weight
layer1.1.bias
layer1.1.running_mean
layer1.1.running_var
layer1.1.num_batches_tracked
layer2.0.weight
layer2.0.bias
layer2.1.weight
layer2.1.bias
layer2.1.running_mean
layer2.1.running_var
layer2.1.num_batches_tracked
layer3.0.weight
layer3.0.bias
layer3.1.weight
layer3.1.bias
layer3.1.running_mean
layer3.1.running_var
layer3.1.num_batches_tracked
layer4.0.weight
layer4.0.bias
layer4.1.weight
layer4.1.bias
layer4.1.running_mean
layer4.1.running_var
layer4.1.num_batches_tracked
layer5.0.weight
layer5.0.bias
layer5.1.weight
layer5.1.bias
layer5.1.running_mean
layer5.1.running_var
layer5.1.num_batches_tracked
fc.1.weight
fc.1.bias
fc1.1.weight
fc1.1.bias
fc2.0.weight
fc2.0.bias


c:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:1802: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [33]:
print(model.state_dict()['layer1.0.bias'])

tensor([-0.0875, -0.0417,  0.1699, -0.1363,  0.0066,  0.0185,  0.1365,  0.0514,
        -0.0012,  0.1076, -0.0461,  0.0020,  0.1244,  0.1618, -0.0557, -0.0742],
       device='cuda:0')


In [44]:

state_dict = model.state_dict()

print(state_dict['layer1.1.weight'])

state_dict['layer1.1.weight'] = state_dict['layer1.1.weight'].float() / 2
print(state_dict['layer1.1.weight'])

tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250,
        0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
       device='cuda:0')
tensor([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
        0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625],
       device='cuda:0')


In [10]:
model = CNN()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=config['momentum'])

In [2]:
from train import Client, Server

In [3]:
config = dict(
	epochs=2,
	batch_size=16,
	criterion='BinaryCrossEntropy', # CrossEntropy, BinaryCrossEntropy
	optimizer='SGD', # SGD
	learning_rate=0.001,
	momentum=0.9,
	dataset='Plant',
	image_type='color', # color, side
	data_augmentation=True,
	image_height=480, # 480, 960
	image_width=640, # 640, 1280
)

In [4]:
client_directories = ['A1', 'A2', 'A4', 'A6', 'B1', 'B2', 'B3', 'B4']
clients = []
for client_dir in client_directories:
	clients.append(Client(client_dir, config))

In [7]:
model_state_dict_total, optimizer_state_dict_total = clients[0].send_to_server()

for client in clients[1:]:
	model_state_dict, optimizer_state_dict = client.send_to_server()

	for layer in model_state_dict:
		model_state_dict_total[layer] = model_state_dict_total[layer].float() + model_state_dict[layer]

In [6]:
model_state_dict_total, optimizer_state_dict_total = clients[0].send_to_server()

In [ ]:
for layer in model_state_dict_total: